In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=13F&dateb=&owner=exclude&count=100"

In [4]:
ticker = '0000908551'

In [7]:
url.format(1344)

'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1344&type=13F&dateb=&owner=exclude&count=100'

In [8]:
resp = requests.get(url.format(ticker))

In [10]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [11]:
soup.prettify()

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html lang="ENG">\n <head>\n  <title>\n   EDGAR Search Results\n  </title>\n  <link href="/include/interactive.css" rel="stylesheet" type="text/css"/>\n  <link href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0000908551&amp;type=13F%25&amp;dateb=&amp;owner=exclude&amp;count=100&amp;output=atom" rel="alternate" title="ATOM" type="application/atom+xml"/>\n </head>\n <body style="margin: 0">\n  <!-- SEC Web Analytics - For information please visit: http://www.sec.gov/privacy.htm#collectedinfo -->\n  <noscript>\n   <iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV" style="display:none;visibility:hidden" width="0">\n   </iframe>\n  </noscript>\n  <script>\n   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=t

In [12]:
url.format(ticker)

'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000908551&type=13F&dateb=&owner=exclude&count=100'

# Find Elements by table Data

In [14]:
table_data = soup.findAll('table')[-1]

In [15]:
table_data

<table class="tableFile2" summary="Results">
<tr>
<th scope="col" width="7%">Filings</th>
<th scope="col" width="10%">Format</th>
<th scope="col">Description</th>
<th scope="col" width="10%">Filing Date</th>
<th scope="col" width="15%">File/Film Number</th>
</tr>
<tr>
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/908551/000090855118000006/0000908551-18-000006-index.htm" id="documentsbutton"> Documents</a></td>
<td class="small">Quarterly report filed by institutional managers, Holdings<br/>Acc-no: 0000908551-18-000006 (34 Act)  Size: 8 KB            </td>
<td>2018-10-29</td>
<td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=028-13140&amp;owner=exclude&amp;count=100">028-13140</a><br/>181143301         </td>
</tr>
<tr class="blueRow">
<td nowrap="nowrap">13F-HR</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/908551/000090855118000005/0000908551-18-000005-index.htm" id="documentsbutton"> Documents</a></td>
<td clas

### Find Elements by class Name of results table

In [21]:
table_data = soup.find('table', class_='tableFile2')

### Find the elements table rows

In [24]:
table_rows = table_data.find_all('tr')

In [29]:
first_filing = table_rows[1]

In [36]:
for td in first_filing.find_all('td'):
    print(td.text)

13F-HR
 Documents
Quarterly report filed by institutional managers, HoldingsAcc-no: 0000908551-18-000006 (34 Act)  Size: 8 KB            
2018-10-29
028-13140181143301         


### Get the Href from the second table data element and href tag

In [184]:
first_filing.find_all('td')[3].text

'2018-10-29'

In [45]:
filings_url = 'https://www.sec.gov' + first_filing.find_all('td')[1].find('a')['href']

### Make Request to filings page

In [47]:
res = requests.get(filings_url)

In [49]:
soup = BeautifulSoup(res.text, 'html.parser')

In [53]:
table = soup.find('table', class_='tableFile')

In [57]:
len(table.find_all('tr')[1:])

5

In [64]:
last_row = table.find_all('tr')[-1]

### Get the href from the third column of table

In [72]:
last_row.find_all('td')[2].find('a')['href']

'/Archives/edgar/data/908551/000090855118000006/0000908551-18-000006.txt'

### Mkae Request for the file

In [73]:
holdings_file_url = 'https://www.sec.gov' + last_row.find_all('td')[2].find('a')['href']

In [76]:
resp = requests.get(holdings_file_url)

In [80]:
soup = BeautifulSoup(resp.text, 'xml')

In [84]:
info_tables = soup.find('informationTable').find_all('infoTable')

In [86]:
info_tables[1]

<infoTable>
<nameOfIssuer>ADR MOMO INC ADR</nameOfIssuer>
<titleOfClass>COM</titleOfClass>
<cusip>60879B107</cusip>
<value>551</value>
<shrsOrPrnAmt>
<sshPrnamt>12585</sshPrnamt>
<sshPrnamtType>SH</sshPrnamtType>
</shrsOrPrnAmt>
<investmentDiscretion>SOLE</investmentDiscretion>
<votingAuthority>
<Sole>12585</Sole>
<Shared>0</Shared>
<None>0</None>
</votingAuthority>
</infoTable>

In [100]:
for info in info_tables[1]:
    if info.name:
        print(info.name, info.text)

nameOfIssuer ADR MOMO INC ADR
titleOfClass COM
cusip 60879B107
value 551
shrsOrPrnAmt 
12585
SH

investmentDiscretion SOLE
votingAuthority 
12585
0
0



## Use the findchildren method instead of finding elements one by one for faster runtime

In [124]:
for child in info_tables[1].findChildren():
    if child.name and not child.findChildren():
        print(child.name, child.find(text=True, recursive=False))

nameOfIssuer ADR MOMO INC ADR
titleOfClass COM
cusip 60879B107
value 551
shrsOrPrnAmt 

sshPrnamt 12585
sshPrnamtType SH
investmentDiscretion SOLE
votingAuthority 

Sole 12585
Shared 0
None 0


In [173]:
issuers = []

In [172]:
for info_table in info_tables:
    for child in info_table.findChildren():
        if child.name and not child.findChildren():
            print(child.name, child.find(text=True, recursive=False))
    print('>>>>>>>>>>>>>>>>>>>>')
    print()

nameOfIssuer ADR JUMEI INTL HLDG LTD
titleOfClass COM
cusip 48138L107
value 248
sshPrnamt 116246
sshPrnamtType SH
investmentDiscretion SOLE
Sole 116246
Shared 0
None 0
>>>>>>>>>>>>>>>>>>>>

nameOfIssuer ADR MOMO INC ADR
titleOfClass COM
cusip 60879B107
value 551
sshPrnamt 12585
sshPrnamtType SH
investmentDiscretion SOLE
Sole 12585
Shared 0
None 0
>>>>>>>>>>>>>>>>>>>>

nameOfIssuer ADR NOAH HOLDINGS
titleOfClass COM
cusip 65487X102
value 331
sshPrnamt 7853
sshPrnamtType SH
investmentDiscretion SOLE
Sole 7853
Shared 0
None 0
>>>>>>>>>>>>>>>>>>>>

nameOfIssuer ADR PPDAI GROUP INC
titleOfClass COM
cusip 69354V108
value 987
sshPrnamt 183546
sshPrnamtType SH
investmentDiscretion SOLE
Sole 183546
Shared 0
None 0
>>>>>>>>>>>>>>>>>>>>

nameOfIssuer EAGLE PHARMACEUTICALS INC
titleOfClass COM
cusip 269796108
value 1047
sshPrnamt 15097
sshPrnamtType SH
investmentDiscretion SOLE
Sole 15097
Shared 0
None 0
>>>>>>>>>>>>>>>>>>>>

nameOfIssuer HERSHEY COMPANY COM STK USD1
titleOfClass COM
cusip 4278661

In [174]:
for info_table in info_tables:
    issue = {}
    for child in info_table.findChildren():
        if child.name:
            issue[child.name] = child.find(text=True, recursive=False).strip()
    issuers.append(issue)
    print('>>>>>>>>>>>>>>>>>>>>')
    print()

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>

>>>>>>>>>>>>>>>>>>>>



In [168]:
for k,v in issuers[0].items():
    print(k, v.strip())

nameOfIssuer ADR JUMEI INTL HLDG LTD
titleOfClass COM
cusip 48138L107
value 248
shrsOrPrnAmt 
sshPrnamt 116246
sshPrnamtType SH
investmentDiscretion SOLE
votingAuthority 
Sole 116246
Shared 0
None 0


In [138]:
issuers[0]

{'nameOfIssuer': 'ADR JUMEI INTL HLDG LTD',
 'titleOfClass': 'COM',
 'cusip': '48138L107',
 'value': '248',
 'sshPrnamt': '116246',
 'sshPrnamtType': 'SH',
 'investmentDiscretion': 'SOLE',
 'Sole': '116246',
 'Shared': '0',
 'None': '0'}

In [175]:
columns = [child.name for child in info_tables[0].findChildren()]

In [176]:
columns

['nameOfIssuer',
 'titleOfClass',
 'cusip',
 'value',
 'shrsOrPrnAmt',
 'sshPrnamt',
 'sshPrnamtType',
 'investmentDiscretion',
 'votingAuthority',
 'Sole',
 'Shared',
 'None']

In [155]:
import pandas

In [177]:
df = pandas.DataFrame(issuers)

df = df[columns]

df.to_csv('issuers2.tsv', sep='\t')